In [ ]:
# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()

In [2]:
using Distributed
using PyPlot
using NPZ
using JSON
using Statistics

using cevander

Generate specified number of random sequences which are composed of Ising-like spins for each element specie (e.g. 2 for Cr, 1 for Mn, -1 for Co, -2 for Ni) under given composition. No need to generate random sequences when use SQS-based method.

In [69]:
compo = [0.45, 0, 0.2, 0.35] #* specify your composition here: follow the Cr-Mn-Co-Ni order

compo_denote = vec2str(compo)
println("denotion for current composition: $(compo_denote)")

atom_num = 32 #* specify the number of atoms in one supercell (cell size)
gen_num = 10 #* specify the number of random sequences (supercell) to generate

println("number of generated random sequences: $(gen_num)")
println("number of atoms per lattice: $(atom_num)")
mat_ = ele_list_gen(compo, atom_num, gen_num, "int")

println("example for a random sequence: $(mat_[1,:])")

#* save generated sequences
mkpath("examples/mc/embed_sequence")
pth_sav = "examples/mc/embed_sequence/mc_seq_$(compo_denote)_$(atom_num).npy"
println("random sequences saved as $(pth_sav)")
npzwrite(pth_sav, mat_)

denotion for current composition: 4502035
number of generated random sequences: 10
number of atoms per lattice: 32
example for a random sequence: [2.0, -2.0, -1.0, 2.0, -1.0, -2.0, 2.0, -2.0, -2.0, -1.0, 2.0, 2.0, 2.0, 2.0, -1.0, 2.0, -2.0, 2.0, -2.0, -2.0, 2.0, -2.0, -1.0, -1.0, 2.0, -2.0, -2.0, 2.0, -2.0, -2.0, 2.0, 2.0]
random sequences saved as examples/mc/embed_sequence/mc_seq_4502035_32.npy


Based on the **existed random sequences** (can be generated by the cell above), derive the r+k-space correlation function; one can also use SQS-based sequences with specified randomness (not included in current program). The correlation functions are input variables that determine the $MSAD^{1/2}$ as output for each input configuration given coefficient, as $MSAD^{1/2}$ will be deduced in the last cellblock.

In [70]:
atom_num = 32 #* number of atoms in the supercell
compo = [0.45, 0, 0.2, 0.35] #* specify your composition here: follow the Cr-Mn-Co-Ni order

@assert abs(sum(compo)-1) < 0.001 "make sure the atomic contents sum up to 1"
compo_denote = vec2str(compo)

embed_pth = "examples/mc/embed_sequence/mc_seq_$(compo_denote)_$(atom_num).npy"

if !isfile(embed_pth)
    println("embedding sequences not existed, check the composition and cell size")
end

embed_list = npzread(embed_pth)
println("current composition: $(compo_denote) number and size of random sequences: $(size(embed_list))")

vec_mat_r_ = main_mc_r(1:size(embed_list, 1), atom_num, embed_list)
vec_mat_r = hcat(vec_mat_r_...)'
vec_mat_k_ = main_mc_k(atom_num, embed_list)
vec_mat = hcat(vec_mat_r, vec_mat_k_)

mkpath("examples/mc/correlate_func")
npzwrite(joinpath("examples/mc/correlate_func/vec_mat_$(compo_denote)_$(atom_num)_mc.npy"), vec_mat)
println("correlation functions saved as examples/mc/correlate_func/vec_mat_$(compo_denote)_$(atom_num)_mc.npy")

current composition: 4502035 number and size of random sequences: (10, 32)
correlation functions saved as examples/mc/correlate_func/vec_mat_4502035_32_mc.npy


Derive correlation function vectors based on **SQS**, one-to-one mapping for each composition-SQS pair. 

In [30]:
atom_num = 256 #* number of atoms in the supercell
compo_list = Matrix{Float64}([
    # 0.25 0.25 0.25 0.25; # CrMnCoNi
    # 1/3 0 1/3 1/3; # CrCoNi
    0.4 0. 0.3 0.3; # nonequiatomic Cr-Mn-Co-Ni
]) #* specify the composition (list) here: follow the Cr-Mn-Co-Ni order

cprvec_sqs_r = main_sqs_r(atom_num, compo_list)
cprvec_sqs_k = main_sqs_k(atom_num, compo_list)

for i in 1:size(compo_list, 1)
    compo_ = compo_list[i,:]
    compo_denote = vec2str(compo_)
    cprvec_sqs = vcat(cprvec_sqs_r[i], cprvec_sqs_k[i])
    cprvec_sqs = reshape(cprvec_sqs, 1, length(cprvec_sqs))

    pth_sav = "examples/sqs/vec_mat_$(compo_denote)_$(atom_num)_sqs.npy"
    println("save as $(pth_sav)")
    npzwrite(pth_sav, cprvec_sqs)
end

save as examples/sqs/vec_mat_4003030_256_sqs.npy


Evaluate $MSAD^{1/2}$ using the coefficients provided.

In [31]:
coef_ = npzread("data/coef/coef_best_mix_108_192_32_240623.npy")
optim_sdf = npzread("data/coef/chosen_ind_mix_108_192_32_240623.npy")

compo = [0.4, 0, 0.3, 0.3] #* specify your composition here: follow the Cr-Mn-Co-Ni order
atom_num = 256 #* specify the number of atoms in the supercell
@assert abs(sum(compo)-1) < 0.001 "make sure the atomic contents sum up to 1"
compo_denote = vec2str(compo)

#* option1: use atomic configuration-derived correlation functions as input, this is sampling number-dependent and cell size-dependent
# vec_mat = npzread("examples/mc/correlate_func/vec_mat_$(compo_denote)_$(atom_num)_mc.npy")
# msad_mc = eval_mix(coef_, vec_mat[:,optim_sdf.+1])
# print("μ(MSAD¹/²) = $(mean(msad_mc)) pm σ(MSAD¹/²) = $(std(msad_mc)) pm")

#* option2: use virtual SQS-derived correlation functions as input, this is sampling number-independent and cell size-dependent
vec_mat_sqs = npzread("examples/sqs/vec_mat_$(compo_denote)_$(atom_num)_sqs.npy")
msad_sqs = eval_mix(coef_, vec_mat_sqs[:,optim_sdf.+1])[1]
println("SQS-based MSAD¹/²: $(msad_sqs) pm")

SQS-based MSAD¹/²: 7.275920191873097 pm
